[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/10--sql-database/02--sqlite.ipynb)

## SQLite 
[https://www.sqlite.org/](https://www.sqlite.org/)

In [6]:
import sqlite3
from sqlite3 import Error

def connect(db_file):
    """ create a database connection to a SQLite database """
    conn = sqlite3.connect(db_file)
    return conn


In [7]:
!mkdir -p var

In [8]:
DB_FILE = "var/company_db.db"

In [9]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select 1, SQLITE_VERSION();")
    res = c.fetchone()
    print(res)

(1, '3.36.0')


In [10]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("DROP TABLE IF EXISTS Employee")
    c.execute("DROP TABLE IF EXISTS Task")
    res = c.fetchone()
    print(res)

None


In [11]:
create_table = '''
CREATE TABLE IF NOT EXISTS Employee (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name text NOT NULL,
  sallary money NOT NULL
)
'''

In [12]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute(create_table)

In [13]:
insert_sql = '''
INSERT INTO Employee (name, sallary) VALUES (?,?)
''';

In [14]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute(insert_sql, ("Jakub", 2500.00))
    c.execute(insert_sql, ("Michał", 3500.00))
    print(c.lastrowid)

2


In [15]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select * from Employee")
    rows = c.fetchall() 
    for row in rows:
        print(row)


(1, 'Jakub', 2500)
(2, 'Michał', 3500)


In [16]:
create_tasks_table = """
    CREATE TABLE IF NOT EXISTS Task (
        id integer PRIMARY KEY,
        employee_id integer,
        name text NOT NULL,
        priority integer,
        FOREIGN KEY (employee_id) REFERENCES Employee(id)
);
"""

conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute(create_tasks_table)

In [17]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("INSERT INTO Task (employee_id, name, priority) VALUES (?, ?, ?)", (10, "Learn DELETE FROM TABLE sql stmt", 100))

In [18]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select * from Task")
    rows = c.fetchall() 
    for row in rows:
        print(row)

(1, 10, 'Learn DELETE FROM TABLE sql stmt', 100)


Note that Foreign keys constraints are not enabled by default in SQLite, you have to enable them first by the running the following command:

In [19]:
enable_fk_check = '''PRAGMA foreign_keys = ON;'''
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute(enable_fk_check)

In [20]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute("PRAGMA foreign_keys")
    rows = c.fetchone() 
    for row in rows:
        print(row)

1


In [21]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute("INSERT INTO Task (employee_id, name, priority) VALUES (?, ?, ?)", (1, "Learn DELETE FROM TABLE sql stmt", 100))

In [22]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select * from Task")
    rows = c.fetchall() 
    for row in rows:
        print(row)

(1, 10, 'Learn DELETE FROM TABLE sql stmt', 100)
(2, 1, 'Learn DELETE FROM TABLE sql stmt', 100)


In [23]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute("INSERT INTO Task (employee_id, name, priority) VALUES (?, ?, ?)", (100, "Learn DELETE FROM TABLE sql stmt", 100))

IntegrityError: FOREIGN KEY constraint failed

In [24]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("""
        Select * from Employee e 
        JOIN Task t on e.id = t.employee_id;
    """)
    rows = c.fetchall() 
    for row in rows:
        print(row)

(1, 'Jakub', 2500, 2, 1, 'Learn DELETE FROM TABLE sql stmt', 100)
